<a href="https://colab.research.google.com/github/ebiduardo/noteBooks/blob/main/eqtaxRL4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
double wffA(double d, double* x, int i)
//evaluate the welfare function in only one period
{
    double U, x0, x1, x2, x3, x4, pd1;
    int j;
    x0 = x[0];
    x1 = x[1];
    x2 = x[2];
    x3 = x[3];
    x4 = x[4];
    j = 0;
    U = 0;
    U = U + 2 * (1 - d) * log(((1 - d) * (x[j]))) / (p2(j));
    U = U + 2 * (d)*log((d) * (x[j])) / (p2(j));
    while (j < i - 1)
    {
        U = U + 2 * (1 - d) * log(((1 - d) * (x[j] + x0)) / 2) / (p2(j));
        U = U + 2 * (d)*log((d) * (x[j] + x0) / 2) / (p2(j));
        j = j + 1;
    }
    U = U + (1 - d) * log((1 - d) * x[j]) / (p2(j - 1));
    U = U + (d)*log((d)*x[j]) / (p2(j - 1));
    pd1 = pd(j - 1, d);
    U = U * pd1;
    return U;
}


double wffB(double d, double* x, int i)
//evaluate the welfare function in only one period
{
    double U, x0, x1, x2, x3, x4, pd1;
    int j;
    x0 = x[0];
    x1 = x[1];
    x2 = x[2];
    x3 = x[3];
    x4 = x[4];
    j = 0;
    U = 0;
    U = U + 2 * (1 - d) * log(((1 - d) * (x[j]))) / (p2(j));
    U = U + 2 * (d)*log((d) * (x[j])) / (p2(j));
    while (j < i - 1)
    {
        U = U + 2 * (1 - d) * log(((1 - d) * (x[j] + x0)) / 2) / (p2(j));
        U = U + 2 * (d)*log((d) * (x[j] + x0) / 2) / (p2(j));
        j = j + 1;
    }
    U = U + (1 - d) * log((1 - d) * x[j]) / (p2(j - 1));
    U = U + (d)*log((d)*x[j]) / (p2(j - 1));
    pd1 = pd(j - 1, d);
    U = U * pd1;
    return U;
}


In [18]:
%%file eqtaxRL4-2.cpp
// eqtaxRL4-2.cpp : Este arquivo contém a função 'main'. A execução do programa começa e termina ali.
//
//
//  main.cpp
//  eqtaxRL3
//
//  Created by JUAN PABLO GAMA TORRES on 19/08/21.
//
#include <stdlib.h>
#include <iostream>
#include <math.h>
#include <stdio.h>
double wffbc1t(double d, double* x, int i, double tc, double tb, double g, double eta, double rho, double t);


double p2(int j)
{
    double p;
    int i;
    i = 0;
    p = 1;
    while (i < j)
    {
        p = p * 2;
        i++;
    }
    return p;
}

double pd(int j, double d)
{
    double p, d1;
    int i;
    i = 0;
    p = 1;
    d1 = (d + 1);
    d1 = d1 / 2;
    while (i < j)
    {
        p = p * d1;
        i++;
    }
    return p;
}

double wffc(double d, double* x, int i, double t)
//evaluate the welfare function in only one period
{
    double U, x0, x1, x2, x3, x4, pd1;
    int j;
    x0 = x[0];
    x1 = x[1];
    x2 = x[2];
    x3 = x[3];
    x4 = x[4];
    j = 0;
    U = 0;
    U = U + 2 * (1 - d) * log(((1 - d) / (1 + t) * (x[j]))) / (p2(j));
    U = U + 2 * (d)*log((d) * (x[j])) / (p2(j));
    while (j < i - 1)
    {
        U = U + 2 * (1 - d) * log(((1 - d) / (1 + t) * (x[j] + x0)) / 2) / (p2(j));
        U = U + 2 * (d)*log((d) * (x[j] + x0) / 2) / (p2(j));
        j = j + 1;
    }
    U = U + (1 - d) * log((1 - d) / (1 + t) * x[j]) / (p2(j - 1));
    U = U + (d)*log((d)*x[j]) / (p2(j - 1));
    pd1 = pd(j - 1, d);
    U = U * pd1;
    return U;
}


double wffbc(double d, double* x, int i, double tc, double tb)
//evaluate the welfare function in only one period
//AJUSTAR O VALOR DOS RISK LOVER PELO QUADRADO E DIVIDINDO POR DOIS!
{
    double U, x0, x1, x2, x3, x4, pd1;
    int j;
    x0 = x[0];
    x1 = x[1];
    x2 = x[2];
    x3 = x[3];
    x4 = x[4];
    j = 0;
    U = 0;
    U = U + 2 * (1 - d) * log(((1 - d) * (1 - tc) * (x[j]))) / (p2(j));
    U = U + 2 * (d)*log((d) * (1 - tb) * (x[j])) / (p2(j));
    while (j < i - 1)
    {
        U = U + 2 * (1 - d) * log(((1 - d) * (1 - tc) * (x[j] + x0)) / 2) / (p2(j));
        U = U + 2 * (d)*log((d) * (1 - tb) * (x[j] + x0) / 2) / (p2(j));
        j = j + 1;
    }
    U = U + (1 - d) * log((1 - d) * (1 - tc) * x[j]) / (p2(j - 1));
    U = U + (d)*log((d) * (1 - tb) * x[j]) / (p2(j - 1));
    pd1 = pd(j - 1, d);
    U = U * pd1;
    return U;
}



double wffbc1(double d, double* x, int i, double tc, double tb, double g, double eta, double rho)
//evaluate the welfare function in only one period
//AJUSTAR O VALOR DOS RISK LOVER PELO QUADRADO E DIVIDINDO POR DOIS!
{
    double U, x0, x1, x2, x3, x4, pd1;
    int j;
    x0 = x[0];
    x1 = x[1];
    x2 = x[2];
    x3 = x[3];
    x4 = x[4];
    j = 0;
    U = 0;
    U = U + wffbc1t(d, x, i, tc, tb, g, eta, rho, j) * pow(d, j + 1);
    j = j + 1;
    while (j < i )
    {
        U = U + wffbc1t(d, x, i, tc, tb, g, eta, rho, j) * pow(d, j + 1);
        j = j + 1;
    }
    U = U * (1 - d) / rho;
    return U;
}


double wffbc1t(double d, double* x, int i, double tc, double tb, double g, double eta, double rho, double t)
//evaluate the welfare function in only one period
//AJUSTAR O VALOR DOS RISK LOVER PELO QUADRADO E DIVIDINDO POR DOIS!
{
    double U, x0, x1, x2, x3, x4, pd1, V;
    int j;
    x0 = x[0];
    x1 = x[1];
    x2 = x[2];
    x3 = x[3];
    x4 = x[4];
    j = 0;
    U = 0;
    U = U + pow(x[j] * pow(g, t) * (1 - d) * (1 - tc), 1 - d) * pow(x[j] * pow(g, t) * (d) * (1 - tb), d);
    U = pow(U, rho) / (p2(j));
    V = 0;
    while (j < i )
    {
        V = pow(pow(x[j] * pow(g, t) * (1 - d) * (1 - tc), 1 - d) * pow(x[j] * pow(g, t) * (d) * (1 - tb), d),2);
        V = V + pow(pow(x[0] * pow(g, t) * (1 - d) * (1 - tc), 1 - d) * pow(x[0] * pow(g, t) * (d) * (1 - tb), d),2);
        V = sqrt(V / 2);
        U = U + pow(V,rho) / (p2(j));
        j = j + 1;
    }
    U = pow(U, eta / rho)-1;
    return U;
}

int main()
//compute the equilibrium for each state, then, compute the welfare function.
{
    int i, j, n, k, l;
    const int K = 100;
    double d, d1, tc, tc1, tb, tb1, RR, RS, U, U1, x0, x1, x2, x3, g, R20, R10, R01, R25, R610, wtbc, E, F, atbc, eta, rho;
    double x[K + 5], y[K + 5], z1[9], z2[9];
    char T;
    FILE* sol;
    //FILE* par;
    z2[0] = 0;
    z2[1] = 0;
    z2[2] = 0;
    z2[3] = 0;
    z2[4] = 0;
    z2[5] = 0;
    z2[6] = 0;
    z2[7] = 0;
    z2[8] = 0;
    //fopen_s(&sol, "eqtaxRL3solOGtax05-15-90-200-07-07.txt", "w");
    sol = fopen("eqtaxRL3solOGtax05-15-90-200-07-07.txt", "w");
    RR = 9.0;
    RS = 4.00;

    n = K + 1;
    //    FILE* sol;
    //    sol = fopen("/Users/Juanpablogamats/Dropbox/economia_matematica/agg1/eqJS1/solJS1.txt", "w");
    d = 0.5;
    //d1 = 0.5;
    d1 = d;
    d1 = d1 * 1.0;
    tc = 0.01;
    tb = 0.01;
    l = 0;
    eta = 0.7;
    rho = 0.7;
    while (tc < 1.000)
    {
        tb = 0.01;
            printf("  tb = %lf\n", tb);
        while (tb < 1.000)
        {
            E = 0;
            tc1 = tc / (1 - tc);
            tb1 = tb / (1 - tb);
            l = l + 1;
            i = 1;
            atbc = (1 - (1 - d) * (tc)-d * (tb));
            //Comp with w+tx^
            //the averages are the only ones in real terms
            x[K + 2] = RR / 2 + RS;
            x[K + 2] = x[K + 2] / 2;
            wtbc = tc1 * x[K + 2] * (1 - d) * (1 - tc) / atbc;
            wtbc = wtbc + tb1 * x[K + 2] * (d) * (1 - tb) / atbc;
            x[0] = RS + wtbc;
            x[0] = x[0];
            x[K + 4] = RR / 2 + wtbc;
            x[K + 4] = x[K + 4] * atbc;
            x[K + 3] = x[0] * atbc;
            x[2] = RR + wtbc;
            x[2] = x[2];
            x[1] = wtbc;
            x0 = x[0];
            x1 = x[1];
            x2 = x[2];
            x3 = x[3];
            U = wffbc(d1, x, 3, tb, tc);
            g = x[2] / 1;
            E = x[0] + x[1] / 2 + x[2] / 2;
            E = E / 2 * (1 - (1 - d) * (tc)-d * (tb));
            E = E - x[K + 2];
            F = x[K + 2] - (((x[K + 3] + x[K + 4]) / 2));
            //printf("F %lf\n", F);
            //printf("E %lf\n", E);
            i++;
            while (i < n)
            {
                y[K + 2] = (x[K + 3] / atbc * d * (1 - tb) * RS + x[K + 4] / atbc * d * (1 - tb) * RR / 2) / 2;
                wtbc = tc1 * y[K + 2] * (1 - d) * (1 - tc) / atbc;
                wtbc = wtbc + tb1 * y[K + 2] * (d) * (1 - tb) / atbc;
                y[K + 3] = x[K + 3] / atbc * d * (1 - tb) * RS + wtbc;
                y[0] = y[K + 3];
                y[K + 3] = y[K + 3] * atbc;
                y[K + 4] = x[K + 4] / atbc * d * (1 - tb) * RR / 2 + wtbc;
                y[K + 4] = y[K + 4] * atbc;
                //Lb = yb[K+3] - xb[K+3]*d/(1+t1)*RS;
                //Lb = Lb/2;
                //fprintf(sol, " %10.8lf ", y[K+2]);
                //fprintf(sol,"%10.8lf  ", y[0]);
                j = 2;
                y[1] = wtbc;
                //fprintf(sol,"%10.8lf  ", y[1]);
                //Ec = yc[0]+yc[1]/2;
                //Ec = Ec/2;
                //Eb = yb[0]+yb[1]/2;
                //Eb = Eb/2;
                //printf("%d %lf %lf\n",0, yc[0], yb[0]);
                //printf("%d %lf %lf\n",1, yc[1], yb[1]);
                //printf("%lf %lf %lf\n", y[K+2]/x[K+2], yc[K+2]/xc[K+2], yb[K+2]/xb[K+2]);
                E = y[0] + y[1] / 2;
                E = E / 2;
                F = y[K + 2] / ((y[K + 3] + y[K + 4]) / 2);
                //printf("F %lf\n", F);
                g = y[K + 2] / x[K + 2];
                while (j < i + 2)
                {
                    y[j] = RR * d * x[j - 1] * (1 - tb) + wtbc;
                    y[j] = y[j];
                    E = E + y[j] / p2(j + 1);
                    //Eb = Eb + yb[j]/p2(j+1);
                    //if (j+1 == i+2)
                    //{
                        //Ec = Ec + yc[j]/p2(j+1);
                        //Eb = Eb + yb[j]/p2(j+1);
                    //}
                    //printf("%d %lf %lf\n",j, yc[j], yb[j]);
                    //if (y[K+2]>y[j])
                    //{
                        //L = L + t*(y[K+2]-x[j-1]*RR*d)/p2(j+1);
                        //Lc = Lc + t1*(yc[K+2]-xc[j-1]*RR*d)/p2(j+1);
                        //Lb = Lb + t1*(yc[K+2]-xc[j-1]*RR*d)/(1+t1)/p2(j+1);
                    //}
                    //fprintf(sol,"%10.8lf  ", y[j]);
                    j++;
                }
                E = E + y[i + 1] / p2(i + 2);
                //Ec = Ec*(1+d*t1)/(1+t1);
                //Ec = Ec-yc[K+2];
                //Eb = Eb*(1+t1*(1-d))/(1+t1);
                //Eb = Eb-yb[K+2];

                //printf("%d %lf %lf\n",K+4, yc[K+4], yb[K+4]);
                //printf("%d %lf %lf\n",K+3, yc[K+3], yb[K+3]);
                //printf("%d %lf %lf\n",K+2, yc[K+2], yb[K+2]);
                //Ec = yc[K+2]-(yc[K+3]+yc[K+4])/2;
                //Eb = yb[K+2]-(yb[K+3]+yb[K+4])/2;
                //L = L/y[K+2];
                //Lc = Lc/yc[K+2];
                //Lb = Lb/yb[K+2];
                x[K + 2] = y[K + 2]/g;
                x[K + 3] = y[K + 3]/g;
                x[K + 4] = y[K + 4]/g;
                E = E * atbc;
                E = E - y[K + 2];
                //printf("%lf\n",E);
                k = 0;
                //if (i+1 == 3)
                //{
                //    g = y[3]/y[1];
                //    printf( " %10.8lf ", g);
                //}
                //g = 0;
                while (k < i + 2)
                {
                    x[k] = y[k]/g;
                    //fprintf(sol,"%10.8lf  ", y[k]);
                    //printf("%10.8lf  ", y[k]);
                    k++;
                    //if (i+1 > 3 & k < i+1 & k >= 3)
                    //{
                    //    g = g + x[k]/p2(k+1);
                    //}
                    //if (i+1 > 3 & k >= i+1 )
                    //{
                    //    g = g + x[k]/p2(k);
                    //    g = g/x[1];
                    //    printf( " %10.8lf ", g);
                    //}
                }
                U = U + wffbc(d1, x, k, tb, tc);
                //fprintf(sol," %10.8lf ", U);
                //fprintf(sol,"  \n");
                i++;

            }
            U1 = wffbc1(d1, x, k, tb, tc, g, eta, rho);
            R20 = 0;
            R10 = 0;
            R610 = 0;
            R25 = x[4] * 0.02875 + x[5] * 0.015625 + x[6] * 0.005625;
            R01 = 0;
            k = 0;
            while (k < i + 2)
            {
                if (k == 2)
                {
                    R20 = x[k] * 0.075;
                }
                if (k >= 3)
                {
                    R20 = R20 + x[k] / p2(k + 1);
                    if (k == 3)
                    {
                        R10 = x[k] * 0.0375;
                    }
                    if (k >= 4)
                    {
                        R10 = R10 + x[k] / p2(k + 1);
                        if (k == 6)
                        {
                            R01 = x[k] * 0.0021875;
                        }
                        if (k >= 7)
                        {
                            R01 = R01 + x[k] / p2(k + 1);
                        }
                    }
                }
                k++;
            }
            //R25 = R25/x[K+2];
            //R20 = R20/x[K+2];
            //R10 = R10/x[K+2];
            //R01 = R01/x[K+2];
            //R610 = R10 - R25 - R01;
            //Rc25 = Rc25/xc[K+2]/(1+t1)*(1+d*t1);
            //Rc20 = Rc20/xc[K+2]/(1+t1)*(1+d*t1);
            //Rc10 = Rc10/xc[K+2]/(1+t1)*(1+d*t1);
            //Rc01 = Rc01/xc[K+2]/(1+t1)*(1+d*t1);
            //Rc610 = Rc10 - Rc25 - Rc01;
            R25 = R25 / x[K + 2] * atbc;
            R20 = R20 / x[K + 2] * atbc;
            R10 = R10 / x[K + 2] * atbc;
            R01 = R01 / x[K + 2] * atbc;
            R610 = R10 - R25 - R01;
            z1[0] = tb;
            z1[1] = tc;
            z1[2] = U;
            z1[3] = g;
            z1[4] = R25;
            z1[5] = R20;
            z1[6] = R10;
            z1[7] = R01;
            z1[8] = U1;
            if (z1[8] > z2[8])
            {
                j = 0;
                while (j < 9)
                {
                    z2[j] = z1[j];
                    j++;
                }
            }
            //fprintf(sol,"\n%10.8lf %10.8lf %10.8lf   %10.8lf %10.8lf %10.8lf %10.8lf %10.8lf %10.8lf\n\n", tb, tc, U, R20, R10, R01, R25, R610, E);
            fprintf(sol, "\n %10.8lf %10.8lf     %10.8lf %10.8lf %10.8lf    ", tb, tc, U, U1, g);
            fprintf(sol, "%10.8lf %10.8lf %10.8lf  %10.8lf \n", R25, R20, R10, R01);
            //fprintf(sol,"%10.8lf %10.8lf %10.8lf  %10.8lf  \n", Rc25, Rc20, Rc10, Rc01);
            //fprintf(sol," %10.8lf %10.8lf %10.8lf  %10.8lf \n\n", Rb25, Rb20, Rb10, Rb01);
            fprintf(sol, "\n\n\n %10.8lf %10.8lf     %10.8lf %10.8lf    ", z2[0], z2[1], z2[2], z2[3]);
            fprintf(sol, "%10.8lf %10.8lf %10.8lf  %10.8lf \n", z2[4], z2[5], z2[6], z2[7]);
            tb = tb + 0.01;
        }
            printf("  tb = %lf\n", tb);
        tc = tc + 0.01;
        fprintf(sol, "\n\n\n %10.8lf %10.8lf     %10.8lf %10.8lf    ", z2[0], z2[1], z2[2], z2[3]);
        fprintf(sol, "%10.8lf %10.8lf %10.8lf  %10.8lf \n", z2[4], z2[5], z2[6], z2[7]);
        printf("tc = %lf\n", tc);
    }
    fprintf(sol, "\n\n\n %10.8lf %10.8lf     %10.8lf %10.8lf    ", z2[0], z2[1], z2[2], z2[3]);
    fprintf(sol, "%10.8lf %10.8lf %10.8lf  %10.8lf \n", z2[4], z2[5], z2[6], z2[7]);
    printf("\n\nPressione qualquer tecla alfa-numerica e depois");
    printf(" ENTER para terminar.\n");
    //scanf_s("%s", &T);
    return 0;
}


Writing eqtaxRL4-2.cpp


In [19]:
!g++  -Ofast eqtaxRL4-2.cpp -o eqtaxRL4-2
#!./eqtaxRL4-2
!ls -ltr
!head eqtaxRL3solOGtax05-15-90-200-07-07.txt
!tail eqtaxRL3solOGtax05-15-90-200-07-07.txt

total 2140
-rw-r--r-- 1 root root 2107283 Jul  1 17:45 eqtaxRL3solOGtax05-15-90-200-07-07.txt
-rw-r--r-- 1 root root   13631 Jul  1 17:49 juanFGV.cpp
-rwxr-xr-x 1 root root   22872 Jul  1 17:58 juanFGV
-rw-r--r-- 1 root root   13631 Jul  1 18:07 eqtaxRL4-2.cpp
-rwxr-xr-x 1 root root   22880 Jul  1 18:08 eqtaxRL4-2

 0.01000000 0.01000000     6.08622860 2.09200895 2.23919970    0.01249452 0.67743824 0.67345911  0.65798856 



 0.01000000 0.01000000     6.08622860 2.23919970    0.01249452 0.67743824 0.67345911  0.65798856 

 0.02000000 0.01000000     6.30044424 2.82791829 2.22279866    0.01857938 0.72934067 0.72338862  0.70036427 



 0.01000000 0.99000000     -15.91832499 4.22195944    0.12544009 0.35283106 0.23615318  0.04587170 



 0.01000000 0.99000000     -15.91832499 4.22195944    0.12544009 0.35283106 0.23615318  0.04587170 



 0.01000000 0.99000000     -15.91832499 4.22195944    0.12544009 0.35283106 0.23615318  0.04587170 
